# Nework Programming, Part 3

## Server의 특성
1. 서버는 중단없이 무한히 돌아가야 한다. Exception이 발생했을 때 복구 가능하거나 무시할 수 있는 것이라면 적절히 exception handling하여 계속 실행되도록 한다.
1. Client 마다 connected socket이 생긴다. 이를 통해 client와 통신한다.
1. 따라서, 서버는 listening socket 1개, client와 연결된 connected n개를 처리해야 한다. 다음과 같은 approach를 적용한다.
   1. Multi-threading 이용: main thread는 listening socket으로 `accept` 처리, connected socket 마다 동일한 function(or method)를 실행시키는 n개의 thread
   2. Concurrent process 이용 - `fork`
   3. I/O multiplexing 이용: n+1개의 socket에 대해 readable(데이터 도착했는지) event가 발생했는지 확인하고, 발생된 socket들에 대해 처리. 보통 non-blocking mode의 socket을 처리.
1. 서버는 보통 컴퓨터가 booting할 때 서버 process가 daemon process로 실행된다. 
Daemon process라 함은 그 컴퓨터의 user가 없고 terminal도 없다. stdout으로 print 불가능.
1. 실행 중 기록을 통상 log 파일에 남기고 and/or 운영자 console로 출력한다. `logging` module 사용

참고:
- [logging HOWTO](https://docs.python.org/ko/3/howto/logging.html)
- [threading — Manage Concurrent Operations Within a Process](https://pymotw.com/3/threading/index.html)
- [selectors — I/O Multiplexing Abstractions](https://pymotw.com/3/selectors/)

## Multi-threading server
Client와 connection이 성립되면 이 client와의 데이터 교환을 책임질 function을 target으로 하는 thread를 start시킨다. 

In [ ]:
while True:
    conn, cli_addr = sock.accept()  # wait for next client connect
    handler = threading.Thread(target=echo_handler, args=(conn, cli_addr))
    handler.start()

> `threading.Thread`는 class instance(object)만 생성하고 실제 thread는 `start` method로 생성되고 시작된다. 이 thread context에서 `echo_handler(args)`가 call된다.

### servers/server_thread.py:
`echo_server` function은 listening socket으로 client의 connection을 accept하고, 
connected socket을 통한 데이터 송수신을 전담할 thread를 start시킨다.

`echo_handler` function은 echo service를 제공하는 code를 implement한다. 
Echo service는 수신한 bytes를 그대로 돌려주면 되기 때문에 메시지가 무엇이지, 어떤 encoding을 했느지 등
내용을 해석할 필요가 없다.

> 사용중인 port를 `bind`하면 *Address in use* error가 발생한다. 
그럼에도 불구하고 가로채기를 하려면 socket에 option을 다음과 같이 설정한다.
```Python
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
```